<a href="https://colab.research.google.com/github/AbimaelLopez/Diplomado/blob/main/9_Naive_Bayes_ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **9. Naive Bayes**

### Objetivos
Que la o el estudiante entienda los principios básicos del algoritmo de clasificación Naive Bayes y que aprenda a utilizar la clase correspondiente de la biblioteca Sklearn.

Que la o el estudiante aprenda a crear Pipelines que incluyen modelos y que se hagan búsqueda de de hiperparámetros y modelos dentro de GridSearchCV.

### Descripción del procedimiento a seguir
- Se construye un clasificador Naive Bayes desde cero para clasificar el conjunto de datos Iris.
- Se utiliza la clase de Sklearn.
- Se comparan tres clasificadores distintos para un problema de clasificación.
- Se deja como ejercicio comparar clasificadores para un problema de regresón.


### Datos
**Profesora**: Dra. Jessica Beltrán Márquez<br>
Maestría en Ciencia de Datos y Optimización<br>
Centro de Investigación en Matemáticas Aplicadas <br>
Universidad Autónoma de Coahuila


### Bibliografía
1. http://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html
2. https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/master/data/Boston.csv


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1. Importar bibliotecas**

In [2]:
#Importamos las bibliotecas que vamos a utilizar
import math
import random
import pandas as pd
import numpy as np

#Datasets
from sklearn.datasets import load_iris

#Model selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold

#Metrics
from sklearn.metrics import accuracy_score

#Classfier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer

## **2. Un clasificador Naive Bayes desde cero**

### **2.1 Carga y separación del conjunto de datos Iris**
- Leemos los datos
- Separamos en X y y
- Separamos en datos de entrenamiento y de prueba

In [100]:
# Nombre del archivo csv con el conjunto de datos
#dataset_filename = '/content/drive/MyDrive/Curso ML Diplomado/9. Bayes ingenuo/irisDataset.csv'
# Cargar los datos a un dataframe pandas

iris = load_iris()


# Create a Pandas DataFrame
X = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Add the target variable (species) to the DataFrame
y = pd.Series(iris.target)
#df = pd.read_csv(dataset_filename)
#df.head()

In [85]:
X = df.iloc[:,:-1]
y= df.iloc[:,-1]

In [101]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=10, stratify=y)

### **2.2 En la variable *classes* guardamos los nombres de las clases**

In [102]:
#Obtener las clases de flores a partir de la última columna
classes = y.unique()
print(classes)

[0 1 2]


### **2.3 Definimos una funciones de apoyo**  
- Definimos la función *GaussianParametersFeatures** que obtiene las medias y desviaciones estándares de cada columna de un conjunto de datos.
- Definimos la función *get_likelihood_normal_distribution* para obtener la verosimuilitud.

**Definimos una funcion que obtiene el vector media y desviación estándar dado un conjunto de datos**
- Con el objetivo de probar la función, le pasamos todo el conjunto de datos y nos obtiene las medias y desviaciones estándar de cada una de las características.

In [103]:
#Esta función obtiene la media y la desviación estándar del dataframe que recibe.
# Lo hace sobre el primer eje
# Nos sirve para conocer los parámetros de las distribuciones Gaussianas de cada clase
def GaussianParametersFeatures(df):
    means = df.mean(axis = 0)
    stds = df.std(axis = 0)
    return means,stds

In [104]:
# Aplicamos la función sobre nuestro conjunto de datos y obtenemos los promedios y desviaciones estándar
(means,stds) = GaussianParametersFeatures(X)
print('Medias: \n', means)
print('\n')
print('Desviaciones estándar: \n', stds)

Medias: 
 sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64


Desviaciones estándar: 
 sepal length (cm)    0.828066
sepal width (cm)     0.435866
petal length (cm)    1.765298
petal width (cm)     0.762238
dtype: float64


**Definimos una función para obtener la verosimilitud dada una distribución normal**
- Con el objetivo de probar la función, le pasamos un vector de medias y un vector de desviaciones estándar.
- Ya que *Naive Bayes* supone independencia, obtenemos el producto de las verosimilitudes de todas las características.

In [105]:
def get_likelihood_normal_distribution(X,mu=0,sigma=1):
  #Calculo de p(x_w) en cada característica
  p_x_w=np.divide(1,(np.sqrt(2*math.pi)*sigma)) * np.exp(-1/2* np.divide(np.power(X-mu, 2),np.multiply(sigma, sigma)))
  return(p_x_w)

In [106]:
#Este código es solo para probar la función, en este caso con varias características
p_x_w = get_likelihood_normal_distribution([5.5, 3, 5, 1],mu=means,sigma=stds)
print(p_x_w)

sepal length (cm)    0.442095
sepal width (cm)     0.907402
petal length (cm)    0.176443
petal width (cm)     0.505789
dtype: float64


In [107]:
#Debido a que estamos suponiendo independencia
np.prod(p_x_w)

0.035800399490383494

### **2.4 Entrenamiento**

Como se vio, con Naive Bayes se entrena un módelo por cada clase

Este modelo se entrena conociendo las probabilidades a priori y las distribuciones de clase

Veamos un ejemplo con la clase 'setosa'

In [108]:
#Primero extraemos solo la información que corresponde a las flores de tipo setosa
train_set_setosa = X_train.loc[y_train == classes[0]]
print(train_set_setosa)

    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
11                4.8               3.4                1.6               0.2
17                5.1               3.5                1.4               0.3
39                5.1               3.4                1.5               0.2
45                4.8               3.0                1.4               0.3
4                 5.0               3.6                1.4               0.2
40                5.0               3.5                1.3               0.3
7                 5.0               3.4                1.5               0.2
12                4.8               3.0                1.4               0.1
27                5.2               3.5                1.5               0.2
2                 4.7               3.2                1.3               0.2
35                5.0               3.2                1.2               0.2
1                 4.9               3.0                1.4               0.2

In [109]:
#Obtenemos las medias y desviaciones estándar de cada característica, lo que nos sirve para describir la distribución Gaussiana
(means,stds) = GaussianParametersFeatures(train_set_setosa)
print('Medias: \n', means)
print('\n')
print('Desviaciones estándar: \n', stds)

Medias: 
 sepal length (cm)    4.995
sepal width (cm)     3.440
petal length (cm)    1.460
petal width (cm)     0.245
dtype: float64


Desviaciones estándar: 
 sepal length (cm)    0.352246
sepal width (cm)     0.373342
petal length (cm)    0.187835
petal width (cm)     0.095943
dtype: float64


Ahora lo hacemos para las 3 clases distintas: ['setosa' 'versicolor' 'virginica']

Almacenamos el modelo en un diccionario llamado summaries

In [111]:
summaries = dict()
for className in classes:
    #summaries['classNames'] = classes[i]
    data_subset_class = X_train[y_train== className]
    prior = len(data_subset_class)/len(X_train)              #<- Probabilidad priori
    (means,stds) = GaussianParametersFeatures(data_subset_class)
    summaries[className] = (prior,means,stds)

In [112]:
print(summaries)

{0: (0.3333333333333333, sepal length (cm)    4.995
sepal width (cm)     3.440
petal length (cm)    1.460
petal width (cm)     0.245
dtype: float64, sepal length (cm)    0.352246
sepal width (cm)     0.373342
petal length (cm)    0.187835
petal width (cm)     0.095943
dtype: float64), 1: (0.3333333333333333, sepal length (cm)    6.0225
sepal width (cm)     2.7950
petal length (cm)    4.3250
petal width (cm)     1.3475
dtype: float64, sepal length (cm)    0.477970
sepal width (cm)     0.330462
petal length (cm)    0.418636
petal width (cm)     0.198698
dtype: float64), 2: (0.3333333333333333, sepal length (cm)    6.5850
sepal width (cm)     2.9550
petal length (cm)    5.5625
petal width (cm)     2.0200
dtype: float64, sepal length (cm)    0.660439
sepal width (cm)     0.309632
petal length (cm)    0.554093
petal width (cm)     0.287518
dtype: float64)}


### **2.5 Aplicamos Naive Bayes**
- Tenemos el modelo en summaries (las medias y desviaciones estándar de cada clase).
- Definimos una función para evaluar ejemplos.
  - Esta función recibe los ejemplos, el modelo, y el nombre de las clases.
  - Obtiene la verosimilitud.
  - Almacena las probabilidades por cada una de las clases
  - Regresa la clase con la probabilidad mayor

In [113]:
#Aplicamos bayes
probs = dict()
def evaluateSample(sample,summaries,classes):
    for className in classes:
        (prior,means,stds) = summaries[className]
        p_x_w = get_likelihood_normal_distribution(sample,mu=means,sigma=stds)
        probs[className] = np.prod(p_x_w)*prior
    return max(probs, key=probs.get)


In [114]:
#Evaluamos sobre un vector
evaluateSample([5, 2, 2, 0.3],summaries,classes)


0

In [115]:
#Evaluar un solo ejemplo del testdata
sample=np.array(X_test.iloc[15])
evaluateSample(sample,summaries,classes)


0

In [117]:
y_test[15]

0

In [118]:
#Evaluar todos los ejemplos del testdata
for row in range(X_test.shape[0]):
    sample=np.array(X_test.iloc[row])
    class_sample = y_test.iloc[row]
    class_predicted = evaluateSample(sample,summaries,classes)
    print("Real: ", class_sample, " | " , class_predicted, ": Predicted")

Real:  1  |  1 : Predicted
Real:  0  |  0 : Predicted
Real:  1  |  1 : Predicted
Real:  2  |  2 : Predicted
Real:  1  |  1 : Predicted
Real:  2  |  2 : Predicted
Real:  0  |  0 : Predicted
Real:  2  |  2 : Predicted
Real:  2  |  2 : Predicted
Real:  0  |  0 : Predicted
Real:  0  |  0 : Predicted
Real:  1  |  1 : Predicted
Real:  2  |  2 : Predicted
Real:  2  |  2 : Predicted
Real:  1  |  1 : Predicted
Real:  0  |  0 : Predicted
Real:  0  |  0 : Predicted
Real:  1  |  1 : Predicted
Real:  2  |  2 : Predicted
Real:  0  |  0 : Predicted
Real:  2  |  2 : Predicted
Real:  2  |  2 : Predicted
Real:  2  |  2 : Predicted
Real:  0  |  0 : Predicted
Real:  0  |  0 : Predicted
Real:  1  |  1 : Predicted
Real:  1  |  1 : Predicted
Real:  0  |  0 : Predicted
Real:  1  |  1 : Predicted
Real:  1  |  1 : Predicted


## **3. Uso del modelo GaussianNB de Sklearn**
- Instanciamos el modelo
- Ajustamos el modelo sobre los datos de entrenamiento
- Predecimos usando el conjunto de prueba
- Evaluamos

In [119]:
# Create and fit the Gaussian Naive Bayes classifier
naive_bayes_classifier = GaussianNB()
naive_bayes_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = naive_bayes_classifier.predict(X_test)

# Evaluate the model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 1.00


## **4. Comparación de desempeño de modelos e hiperparámetros para clasificar el conjunto de datos Titanic**

### **4.1 Carga y separación de datos**

In [120]:
# Load the Titanic dataset (replace the path with your actual file path)
titanic_data = pd.read_csv('/content/titanic (1).csv')

# Drop irrelevant columns
titanic_data = titanic_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Separate features and target variable
X = titanic_data.drop('Survived', axis=1)
y = titanic_data['Survived']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



### **4.2 Definición de Pipeline para preprocesamiento de datos numéricos y categóricos**

In [121]:
# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create column transformer to apply transformations to numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


### **4.3 Instanciamiento de tres clasificadores**

In [122]:
# each of these models will take a turn as the second Pipeline step
clf1 = CategoricalNB()
clf2 = KNeighborsClassifier()
clf3 = DecisionTreeClassifier()

### **4.4 Creación de Pipeline con preprocesamiento y clasificador**
NOTA: Se deja un clasificador como *placeholder*, pero esto se sobreescribe con los parámetros definindos.

In [123]:
# create the Pipeline
pipe = Pipeline([('preprocessor', preprocessor), ('classifier', clf1)])

### **4.5 Se definen diccionarios de hyperparametros para cada modelo**

In [124]:

# create the parameter dictionary for clf1
params1 = {}
params1['classifier__alpha'] = [1.0, 2.0]
params1['classifier'] = [clf1]

# create the parameter dictionary for clf2
params2 = {}
params2['classifier__n_neighbors'] =  [3, 5, 7]
params2['classifier__weights'] = ['uniform', 'distance']
params2['classifier'] = [clf2]

# create the parameter dictionary for clf2
params3 = {}
params3['classifier__criterion'] = ['gini', 'entropy']
params3['classifier__max_depth'] = [None, 5, 10, 15]
params3['classifier'] = [clf3]


### **4.6 Creación de lista de diccionarios de parámetros**

In [125]:
# create a list of parameter dictionaries
params = [params1, params2, params3]

### **4.7 Busqueda de mejores modelos con *GridSearchCV***

In [126]:
# this will search every parameter combination within each dictionary
grid = GridSearchCV(pipe, params)
grid.fit(X_train, y_train)
grid.best_params_
print(grid.best_params_)

{'classifier': DecisionTreeClassifier(max_depth=5), 'classifier__criterion': 'gini', 'classifier__max_depth': 5}


### **4.8 Acceso al mejor modelo y evaluación en conjunto de prueba**

In [127]:
# Access the best model (best estimator) directly
best_model = grid.best_estimator_
best_model_name = best_model.named_steps['classifier'].__class__.__name__
# Now you can use the best_model for further analysis or evaluation
best_predictions = best_model.predict(X_test)
best_accuracy = accuracy_score(y_test, best_predictions)

print(f"Best Model Test Accuracy: {best_accuracy}")

print(f"Best Model: {best_model_name}")
print(f"Best Model parameters: {best_model['classifier'].get_params()}")


Best Model Test Accuracy: 0.7653631284916201
Best Model: DecisionTreeClassifier
Best Model parameters: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}


## **5. Ejercicio. Comparación de modelos con problema de regresión**
- Se comparan los modelos de regresión de KNN y Decisión Tree para regresión.
- Se usa el conjunto de datos Boston House
- La variable a predecir es 'MEDV'
- Crear un pipeline para transformación de columnas numéricas
- Crear un pipeline con preprocesador y regresor (como placeholder)
- Definir una lista de diccionario de parametros y regresores.
- Encontrar el mejor modelo con GridSearchCV
- Evaluar en el conjunto de prueba.

NOTA: No olvidar hacer las importaciones y cambios necesarios para trabajar con el problema de regresión.

-https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/master/data/Boston.csv

In [158]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
#Metrics
from sklearn.metrics import r2_score

In [128]:
url = "https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/master/data/Boston.csv"
#column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS','NOX', 'RM', 'AGE', 'DIS', 'RAD', 'RAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
#boston_data = pd.read_csv(url, names=column_names)

boston_data = pd.read_csv(url)

boston_data.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [129]:
# Select features and target variable
X = boston_data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']]
y = boston_data['MEDV']
X.describe()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000


In [130]:
rows_with_nan = X[X.isna().any(axis=1)]
print("Rows with NaN values:\n", rows_with_nan)

Rows with NaN values:
 Empty DataFrame
Columns: [CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, LSTAT]
Index: []


In [131]:
# Split the data into training and testing sets using stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [135]:
# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['float64']).columns
categorical_features = X.select_dtypes(include=['int64']).columns

# Create preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create column transformer to apply transformations to numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [145]:
# each of these models will take a turn as the second Pipeline step
rg1 = DecisionTreeRegressor()
rg2 = KNeighborsRegressor()

In [146]:
# create the Pipeline
pipe = Pipeline([('preprocessor', preprocessor), ('regressor', rg1)])

In [153]:

# create the parameter dictionary for clf1
params1 = {}
#params1['classifier__alpha'] = [1.0, 2.0]
params1['regressor__criterion'] = ['squared_error','absolute_error','friedman_mse', 'poisson']
params1['regressor__max_depth'] = [None, 5, 10, 15, 20]
params1['regressor__min_samples_split'] = [2, 5, 10]
params1['regressor__min_samples_leaf']= [ 1, 2, 4,8]
params1['regressor'] = [rg1]

# create the parameter dictionary for clf2
params2 = {}
params2['regressor__n_neighbors'] =  [3, 5, 7]
params2['regressor__weights'] = ['uniform', 'distance']
params2['regressor'] = [rg2]

In [154]:
# create a list of parameter dictionaries
params = [params1, params2]

In [155]:
# this will search every parameter combination within each dictionary
grid = GridSearchCV(pipe, params,scoring='neg_mean_squared_error', error_score='raise')
grid.fit(X_train, y_train)
grid.best_params_
print(grid.best_params_)

{'regressor': KNeighborsRegressor(n_neighbors=3, weights='distance'), 'regressor__n_neighbors': 3, 'regressor__weights': 'distance'}


In [159]:
# Access the best model (best estimator) directly
best_model = grid.best_estimator_
best_model_name = best_model.named_steps['regressor'].__class__.__name__
# Now you can use the best_model for further analysis or evaluation
best_predictions = best_model.predict(X_test)
best_accuracy = r2_score(y_test, best_predictions)

print(f"Best Model Test Accuracy: {best_accuracy}")

print(f"Best Model: {best_model_name}")
print(f"Best Model parameters: {best_model['regressor'].get_params()}")

Best Model Test Accuracy: 0.772119714397104
Best Model: KNeighborsRegressor
Best Model parameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
